In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
out_dir = "./data/out"
in_dir = "./data/in"

In [3]:
optimizer_name = "brute force"
set_name = "set-1"
optimizer_dir = os.path.join(out_dir, optimizer_name)
set_dir = os.path.join(optimizer_dir, set_name)

In [4]:
experiment_dirs = []

for filename in os.listdir(set_dir):
    path = os.path.join(set_dir, filename)
    
    if os.path.isdir(path):
        experiment_dirs.append(path)

In [5]:
experiment_dirs

['./data/out/brute force/set-1/03',
 './data/out/brute force/set-1/04',
 './data/out/brute force/set-1/05',
 './data/out/brute force/set-1/02',
 './data/out/brute force/set-1/10',
 './data/out/brute force/set-1/07',
 './data/out/brute force/set-1/09',
 './data/out/brute force/set-1/08',
 './data/out/brute force/set-1/01',
 './data/out/brute force/set-1/06']

In [6]:
quote_asset_name = "USDT"

asset_col_name = "Měna"
indic_col_name = "Ukazatel"
period_col_name = "Období"
levels_col_name = "Nákupní úrovně (%)"
sizes_col_name = "Nákupní velikosti (%)"
config_overview = pd.DataFrame(columns = [asset_col_name, indic_col_name, period_col_name, levels_col_name,
                                          sizes_col_name])

optimization_criterion = "PROM"
net_profit_col_name = "Zisk (%)"
optim_value_col_name = optimization_criterion
profit_factor_col_name = "Ziskový faktor"
max_close_balance_drawdown_col_name = "Největší sestupný pohyb zůstatku po uzavření pozice (%)"
max_equity_drawdown_col_name = "Největší sestupný pohyb čisté hodnoty (%)"
stats_overview = pd.DataFrame(columns = [asset_col_name, optim_value_col_name, net_profit_col_name,
                                         profit_factor_col_name, max_close_balance_drawdown_col_name,
                                         max_equity_drawdown_col_name])

In [7]:
def ftos(num):
    out = "{:,.1f}".format(num).replace(',', ' ')
    return out.replace(".", ",")

In [8]:
def rtof(frac):
    num, denom = frac.split("/")
    return float(num)/float(denom)

In [9]:
def list_to_string(arr):
    begin = r"\([\begin{array}{" + "c"*len(arr) + "} " 
    end = r"\end{array}]\)"
    out = begin
    
    for i in range(len(arr)-1):
        out += arr[i] + " & "
    
    out += arr[-1] + " " + end
    return out

In [10]:
def convert_levels(levels):
    out_levels = []
    
    for level in levels:
        out_levels.append(ftos((rtof(level)-1.)*100))
        
    return list_to_string(out_levels)

In [11]:
def convert_sizes(sizes):
    out_sizes = []
    
    for size in sizes:
        out_sizes.append(ftos(rtof(size)*100))
        
    return list_to_string(out_sizes)

In [12]:
init_balance = 10_000

for experiment_dir in experiment_dirs:
    settings_path = os.path.join(experiment_dir, "settings.json")
    best_states_path = os.path.join(experiment_dir, "best-states.json")
    
    settings_file = open(settings_path, "r")
    settings_doc = json.loads(settings_file.read())
    
    asset_name = settings_doc["candles"]["currency pair"]["base"]
    
    best_states_file = open(best_states_path, "r")
    best_states_doc = json.loads(best_states_file.read())
    best_state_doc = best_states_doc[0]
    
    best_config = best_state_doc["configuration"]
    indic_doc = best_config["indicator"]
    indic_name = indic_doc["type"].upper()
    indic_period = indic_doc["period"]
    levels = convert_levels(best_config["levels"])
    sizes = convert_sizes(best_config["open sizes"])
    
    config_overview.loc[len(config_overview)] = {
        asset_col_name : asset_name,
        indic_col_name : indic_name,
        period_col_name : indic_period,
        levels_col_name : levels,
        sizes_col_name : sizes,
    }
    
    optim_value = ftos(best_state_doc["optimization value"])
    best_stats = best_state_doc["statistics"]
    net_profit = ftos((best_stats["net profit"]/init_balance)*100)
    profit_factor = best_stats["profit factor"]
    
    if profit_factor is None:
        profit_factor = np.inf
    else:
        profit_factor = ftos(profit_factor)
    
    max_close_balance_drawdown = ftos(best_stats["close balance"]["max drawdown"]["percent"])
    max_equity_drawdown = ftos(best_stats["equity"]["max drawdown"]["percent"])
    
    stats_overview.loc[len(stats_overview)] = {
        asset_col_name : asset_name,
        optim_value_col_name : optim_value,
        net_profit_col_name : net_profit,
        profit_factor_col_name : profit_factor,
        max_close_balance_drawdown_col_name : max_close_balance_drawdown,
        max_equity_drawdown_col_name : max_equity_drawdown,
    }

In [13]:
# sort rows
config_overview = config_overview.sort_values(asset_col_name)
stats_overview = stats_overview.sort_values(asset_col_name)

In [14]:
config_overview

,Měna,Ukazatel,Období,Nákupní úrovně (%),Nákupní velikosti (%)
8,ADA,EMA,51,"\([\begin{array}{ccc} -14,1 & -17,2 & -20,3 \e...","\([\begin{array}{ccc} 12,5 & 12,5 & 75,0 \end{..."
3,BTC,SMA,15,"\([\begin{array}{ccc} -9,4 & -10,9 & -12,5 \en...","\([\begin{array}{ccc} 12,5 & 75,0 & 12,5 \end{..."
0,DASH,SMA,15,"\([\begin{array}{ccc} -14,1 & -17,2 & -18,8 \e...","\([\begin{array}{ccc} 12,5 & 75,0 & 12,5 \end{..."
1,DOGE,SMA,15,"\([\begin{array}{ccc} -12,5 & -20,3 & -21,9 \e...","\([\begin{array}{ccc} 25,0 & 12,5 & 62,5 \end{..."
2,ETH,SMA,18,"\([\begin{array}{ccc} -14,1 & -15,6 & -17,2 \e...","\([\begin{array}{ccc} 75,0 & 12,5 & 12,5 \end{..."
9,LTC,EMA,12,"\([\begin{array}{ccc} -10,9 & -12,5 & -14,1 \e...","\([\begin{array}{ccc} 12,5 & 12,5 & 75,0 \end{..."
5,SOL,EMA,9,"\([\begin{array}{ccc} -7,8 & -12,5 & -17,2 \en...","\([\begin{array}{ccc} 12,5 & 62,5 & 25,0 \end{..."
7,XLM,SMA,60,"\([\begin{array}{ccc} -12,5 & -17,2 & -23,4 \e...","\([\begin{array}{ccc} 12,5 & 12,5 & 75,0 \end{..."
6,XRP,SMA,9,"\([\begin{array}{ccc} -10,9 & -12,5 & -21,9 \e...","\([\begin{array}{ccc} 12,5 & 12,5 & 75,0 \end{..."
4,ZRX,SMA,15,"\([\begin{array}{ccc} -14,1 & -15,6 & -21,9 \e...","\([\begin{array}{ccc} 12,5 & 62,5 & 25,0 \end{..."


In [15]:
stats_overview

,Měna,PROM,Zisk (%),Ziskový faktor,Největší sestupný pohyb zůstatku po uzavření pozice (%),Největší sestupný pohyb čisté hodnoty (%)
8,ADA,"754,8","913,4","65,9","-6,4","-26,4"
3,BTC,"391,0","512,8","5,3","-26,7","-46,7"
0,DASH,"138,1","185,3","13,4","-9,8","-40,0"
1,DOGE,"1 309,3","1 541,7","35,5","-4,7","-22,2"
2,ETH,"664,5","940,5","5,1","-8,5","-47,4"
9,LTC,"739,8","885,6","14,9","-6,2","-38,5"
5,SOL,"202,5","262,8","3,5","-16,7","-33,2"
7,XLM,"581,3","674,5","947,8","-0,3","-42,5"
6,XRP,"225,2","292,4","5,9","-18,3","-24,9"
4,ZRX,"268,3","326,7","165,5","-0,3","-36,7"


In [16]:
report_dir = "./data/report"
optimizer_dir = os.path.join(report_dir, optimizer_name)

if not os.path.exists(optimizer_dir):
    os.makedirs(optimizer_dir)

set_dir = os.path.join(optimizer_dir, set_name)

if not os.path.exists(set_dir):
    os.makedirs(set_dir)

stats_overview.to_csv(os.path.join(set_dir, "statistics-overview.csv"), index=False)
config_overview.to_csv(os.path.join(set_dir, "config-overview.csv"), index=False)